In [1]:
%pip install anthropic requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 4.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from anthropic import Anthropic
import requests
from bs4 import BeautifulSoup
import json
import os
from dotenv import load_dotenv

load_dotenv()

api_key=os.getenv("ANTHROPIC_API_KEY")
client = Anthropic(api_key=api_key)
MODEL_NAME = "claude-3-haiku-20240307"



In [27]:
tools = [
    {
        "name": "print_summary",
        "description": "Prints a summary of the article.",
        "input_schema": {
            "type": "object",
            "properties": {
                "author": {"type": "string", "description": "Name of the article author"},
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of topics, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.'
                },
                "summary": {"type": "string", "description": "Summary of the article. One or two paragraphs max."},
                "coherence": {"type": "integer", "description": "Coherence of the article's key points, 0-100 (inclusive)"},
                "persuasion": {"type": "number", "description": "Article's persuasion score, 0.0-1.0 (inclusive)"}
            },
            "required": ['author', 'topics', 'summary', 'coherence', 'persuasion']
        }
    }
]

url = "https://www.anthropic.com/news/third-party-testing"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
article = " ".join([p.text for p in soup.find_all("p")])

query = f"""
<article>
{article}
</article>

Use the `print_summary` tool.
"""

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
    
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_summary":
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")

JSON Summary:
{
  "author": "Anthropic",
  "topics": [
    "ai",
    "regulation",
    "policy",
    "testing",
    "safety"
  ],
  "summary": "The article argues that the AI sector needs effective third-party testing for frontier AI systems to avoid societal harm, whether deliberate or accidental. It discusses what third-party testing looks like, why it's needed, and some of the research Anthropic has done to arrive at this policy position. The article states that developing a robust, third-party testing regime is one of the best ways to manage the challenges of AI today and for the future. It outlines key ingredients of such a regime, including testing for national security risks. The article also discusses Anthropic's Responsible Scaling Policy, while arguing that self-governance by individual companies is insufficient and that industry-wide third-party testing is needed. The article concludes by outlining Anthropic's plans to support governments in developing effective third-party 

In [35]:
response.content[1].input

{'author': 'Anthropic',
 'topics': ['ai', 'regulation', 'policy', 'testing', 'safety'],
 'summary': "The article argues that the AI sector needs effective third-party testing for frontier AI systems to avoid societal harm, whether deliberate or accidental. It discusses what third-party testing looks like, why it's needed, and some of the research Anthropic has done to arrive at this policy position. The article states that developing a robust, third-party testing regime is one of the best ways to manage the challenges of AI today and for the future. It outlines key ingredients of such a regime, including testing for national security risks. The article also discusses Anthropic's Responsible Scaling Policy, while arguing that self-governance by individual companies is insufficient and that industry-wide third-party testing is needed. The article concludes by outlining Anthropic's plans to support governments in developing effective third-party testing regimes.",
 'coherence': 90,
 'pers

In [36]:
tools = [
    {
        "name": "print_all_characteristics",
        "description": "Prints all characteristics which are provided.",
        "input_schema": {
            "type": "object",
            "additionalProperties": True
        }
    }
]

query = f"""Given a description of a character, your task is to extract all the characteristics of the character and print them using the print_all_characteristics tool.

The print_all_characteristics tool takes an arbitrary number of inputs where the key is the characteristic name and the value is the characteristic value (age: 28 or eye_color: green).

<description>
The man is tall, with a beard and a scar on his left cheek. He has a deep voice and wears a black leather jacket.
</description>

Now use the print_all_characteristics tool."""

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_all_characteristics"},
    messages=[{"role": "user", "content": query}]
)

tool_output = None
json_classification=None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_all_characteristics":
        tool_output = content.input
        break

if tool_output:
    print("Characteristics (JSON):")
    print(json.dumps(tool_output, indent=2))
else:
    print("Something went wrong.")

Characteristics (JSON):
{
  "height": "tall",
  "facial_hair": "beard",
  "scars": "left cheek",
  "voice": "deep",
  "attire": "black leather jacket"
}
